<a href="https://colab.research.google.com/github/Charanjeev-2k2/Writing-Assistant-Essay-Prediction/blob/main/Essay_Text_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import tensorflow 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

#Data Preprocessing

In [ ]:
import string
file = open("Essays for text prediction model.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
  
data = ""

for i in lines:
    data = ' '. join(lines)

#Replace \n,\r,\ufeff with spaces   
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

#Replace punctuation with spaces
translator_string = data.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator_string)

#Removing repeating words
help = []
for i in new_data.split():
    if (i not in help):
        help.append(i)

# help = new_data.split()
# help = list(set(help))
data = ' '.join(help)

In [ ]:
#TOKENIZE


tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[1, 65, 1017, 66, 67, 1018, 1019, 68, 1020, 2]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

sequences = np.array(sequences)
type(sequences[-2][0])

numpy.int64

In [ ]:
X = []
y = []

for i in sequences:
      X.append(i[0])
      y.append(i[1])
    
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=vocab_size)

y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#Model Creation

In [ ]:
#Creating the model
import keras
from keras import layers 
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 10, input_length=1))
model.add(keras.layers.LSTM(256, return_sequences=True))
model.add(keras.layers.LSTM(256))
model.add(keras.layers.Dense(1000, activation="tanh"))
model.add(layers.Dense(vocab_size, activation="softmax"))



In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             117670    
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 256)            273408    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              257000    
_________________________________________________________________
dense_5 (Dense)              (None, 11767)             11778767  
Total params: 12,952,157
Trainable params: 12,952,157
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Implementing callbacks
import tensorflow 
from tensorflow import keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
from keras import optimizers
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.01))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/50
201/201 [==============================] - 41s 184ms/step - loss: 9.4466

Epoch 00001: loss improved from inf to 9.54594, saving model to nextword1.h5
Epoch 2/50
201/201 [==============================] - 37s 183ms/step - loss: 9.5614

Epoch 00002: loss improved from 9.54594 to 9.38086, saving model to nextword1.h5
Epoch 3/50
201/201 [==============================] - 36s 180ms/step - loss: 8.4813

Epoch 00003: loss improved from 9.38086 to 8.56670, saving model to nextword1.h5
Epoch 4/50
201/201 [==============================] - 36s 180ms/step - loss: 7.2435

Epoch 00004: loss improved from 8.56670 to 7.52159, saving model to nextword1.h5
Epoch 5/50
201/201 [==============================] - 36s 179ms/step - loss: 6.2872

Epoch 00005: loss improved from 7.52159 to 6.64474, saving model to nextword1.h5
Epoch 6/50
201/201 [==============================] - 36s 179ms/step - loss: 5.4679

Epoch 00006: loss improved from 6.64474 to 5.85986, saving model to nextword1.h5
Epoch 7/

In [ ]:
loss = model.evaluate(X, y)
loss

402/402 [==============================] - 12s 31ms/step - loss: 0.1570


0.15703359246253967

#Model Prediction

In [ ]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword1.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [ ]:


"""
    Testing our model until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

while(True):

    text = input("Enter your line: ")
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue

Enter your line: this


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


might
Enter your line: what
regarded
Enter your line: where
shallower
Enter your line: that
distinction
Enter your line: stop the script
Ending The Program.....
